### Data MIning
### Decision Trees usando h2o
### Edgar Acuna
### Noviembre 2019

In [2]:
import h2o
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn import tree
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import graphviz
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
#h2o.connect()
#h2o.no_progress()
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323 . connected.


H2O cluster uptime:,23 mins 41 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.27.0.4835
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_eacun_4jblcp
H2O cluster total nodes:,1
H2O cluster free memory:,3.320 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


### I Decision Trees para el dataset de Notas usando h2o

In [3]:
#df=h2o.import_file("c://PW-PR//eje1disna.csv",na_strings=[''])
df=h2o.import_file("https://raw.githubusercontent.com/eacunafer/UNMSM2019/master/Datasets/eje1disna.csv",na_strings=[""])

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
df.head(15)

E1,E2,PF,Nota
96,100,100,p
96,94,99,p
100,91,97,p
93,96,97,p
90,94,95,p
75,83,94,p
75,99,94,p
64,93,94,p
89,55,93,p
57,94,92,p


In [5]:
myx=['E1','E2']
df['Nota']=df['Nota'].asfactor()
myy='Nota'

In [6]:
#Como caso particular de gradient boosting
gbm1 = H2OGradientBoostingEstimator(model_id="gbm_covType_v1",ntrees = 1, max_depth=2,sample_rate = 1,col_sample_rate = 1,seed=2000000
)
gbm1.train(myx, myy, training_frame=df)
y_pred=gbm1.predict(df)
print((y_pred['predict']==df['Nota']).sum()/float(len(df)))
gbm1.model_performance(df)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
0.25

ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.16731169195153112
RMSE: 0.4090375190022684
LogLoss: 0.5095972545039459
Mean Per-Class Error: 0.08333333333333326
AUC: 0.9166666666666666
pr_auc: 0.14583333333333331
Gini: 0.8333333333333333

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7741589234136044: 


,,f,p,Error,Rate
0,f,8.0,0.0,0.0,(0.0/8.0)
1,p,4.0,20.0,0.1667,(4.0/24.0)
2,Total,12.0,20.0,0.125,(4.0/32.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.774159,0.909091,0.0
1,max f2,0.706074,0.937500,1.0
2,max f0point5,0.774159,0.961538,0.0
3,max accuracy,0.774159,0.875000,0.0
4,max precision,0.774159,1.000000,0.0
5,max recall,0.706074,1.000000,1.0
6,max specificity,0.774159,1.000000,0.0
7,max absolute_mcc,0.774159,0.745356,0.0
8,max min_per_class_accuracy,0.774159,0.833333,0.0
9,max mean_per_class_accuracy,0.774159,0.916667,0.0



Gains/Lift Table: Avg response rate: 75.00 %, avg score: 74.86 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.625,0.774159,1.333333,1.333333,1.000000,0.774159,1.00,0.774159,0.833333,0.833333,33.333333,33.333333
1,,2,1.000,0.706074,0.444444,1.000000,0.333333,0.706074,0.75,0.748627,0.166667,1.000000,-55.555556,0.000000


In [7]:
#Como caso particular de rabdom forest
model=H2ORandomForestEstimator(ntrees=1)
model.train(myx, myy, training_frame = df)
y_pred=model.predict(df)
print((y_pred['predict']==df['Nota']).sum()/float(len(df)))
model.model_performance(df)

drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
1.0

ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.0
RMSE: 0.0
LogLoss: 0.0
Mean Per-Class Error: 0.0
AUC: 1.0
pr_auc: 0.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 1.0: 


,,f,p,Error,Rate
0,f,8.0,0.0,0.0,(0.0/8.0)
1,p,0.0,24.0,0.0,(0.0/24.0)
2,Total,8.0,24.0,0.0,(0.0/32.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,1.0,1.0,0.0
1,max f2,1.0,1.0,0.0
2,max f0point5,1.0,1.0,0.0
3,max accuracy,1.0,1.0,0.0
4,max precision,1.0,1.0,0.0
5,max recall,1.0,1.0,0.0
6,max specificity,1.0,1.0,0.0
7,max absolute_mcc,1.0,1.0,0.0
8,max min_per_class_accuracy,1.0,1.0,0.0
9,max mean_per_class_accuracy,1.0,1.0,0.0



Gains/Lift Table: Avg response rate: 75.00 %, avg score: 75.00 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.75,1.0,1.333333,1.333333,1.0,1.0,1.00,1.00,1.0,1.0,33.333333,33.333333
1,,2,1.00,0.0,0.000000,1.000000,0.0,0.0,0.75,0.75,0.0,1.0,-100.000000,0.000000


### V. Applying Decision trees to a dataset (adult) containing missing values (using h2o)

In [8]:
#Leyendo los datos
datos= h2o.import_file("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",na_strings=[' ?'])
myx=['C1', 'C2', 'C3', 'C4', 'C5','C6','C7','C8','C9','C10','C11', 'C12','C13','C14']
datos['C15']=datos['C15'].asfactor()
myy="C15"

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
datos.head()

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [10]:
model=H2ORandomForestEstimator(ntrees=1,mtries=14,max_depth=3,sample_rate=1)
model.train(myx, myy, training_frame = datos)
y_pred=model.predict(datos)
print ((y_pred['predict']==datos['C15']).sum()/float(len(datos)))
model.model_performance(datos)

drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
0.8445993673412979

ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.11339590192848084
RMSE: 0.33674308000088266
LogLoss: 0.3559405149152106
Mean Per-Class Error: 0.21962523046025184
AUC: 0.8621853033531734
pr_auc: 0.659144958338928
Gini: 0.7243706067063469

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6744805574417114: 


,,<=50K,>50K,Error,Rate
0,<=50K,23477.0,1243.0,0.0503,(1243.0/24720.0)
1,>50K,3817.0,4024.0,0.4868,(3817.0/7841.0)
2,Total,27294.0,5267.0,0.1554,(5060.0/32561.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.674481,0.613976,3.0
1,max f2,0.200000,0.750275,5.0
2,max f0point5,0.674481,0.695977,3.0
3,max accuracy,0.674481,0.844599,3.0
4,max precision,0.995575,0.995575,0.0
5,max recall,0.025577,1.000000,7.0
6,max specificity,0.995575,0.999879,0.0
7,max absolute_mcc,0.674481,0.537524,3.0
8,max min_per_class_accuracy,0.300194,0.672087,4.0
9,max mean_per_class_accuracy,0.300194,0.780375,4.0



Gains/Lift Table: Avg response rate: 24.08 %, avg score: 24.08 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.020822,0.995575,4.134285,4.134285,0.995575,0.995575,0.995575,0.995575,0.086086,0.086086,313.428450,313.428450
1,,2,0.036854,0.980843,4.073106,4.107672,0.980843,0.980843,0.989167,0.989167,0.065298,0.151384,307.310624,310.767196
2,,3,0.046467,0.974441,4.046521,4.095021,0.974441,0.974441,0.986120,0.986120,0.038898,0.190282,304.652085,309.502140
3,,4,0.161758,0.674481,2.800888,3.172641,0.674481,0.674481,0.764002,0.764002,0.322918,0.513200,180.088781,217.264101
4,,5,0.462946,0.300194,1.246602,1.919579,0.300194,0.300194,0.462253,0.462253,0.375462,0.888662,24.660226,91.957865
5,,6,0.568932,0.148868,0.618507,1.677204,0.148942,0.148942,0.403887,0.403887,0.065553,0.954215,-38.149325,67.720353
6,,7,1.000000,0.025577,0.106213,1.000000,0.025577,0.025577,0.240810,0.240810,0.045785,1.000000,-89.378707,0.000000
